## 1st: Prepping the data

1. Import the library/dataframe and recode/add all required variables
2. Create & export descriptions
2. Create & export a confusion matrix
4. Export the final dataframe

### Import libraries 

In [1]:
#Pandas is a software library written for the Python programming language for data manipulation and analysis.
import pandas as pd
#NumPy is a library for the Python programming language, adding support for large, multi-dimensional arrays and matrices, along with a large collection of high-level mathematical functions to operate on these arrays
import numpy as np
# Matplotlib is a plotting library for python and pyplot gives us a MatLab like plotting framework. We will use this in our plotter function to plot data.
import matplotlib.pyplot as plt
#Seaborn is a Python data visualization library based on matplotlib. It provides a high-level interface for drawing attractive and informative statistical graphics
import seaborn as sns
import dataframe_image as dfi

from matplotlib.ticker import StrMethodFormatter

### Load and view data 

In [2]:
read_file = pd.read_excel (open (r"C:\Users\20204113\OneDrive - TU Eindhoven\2_Research\1_Groceries\DATA\Groceries sorted\DATA GROCERIES-forpredict(HH1-5).xlsx", 'rb'), sheet_name='H3 (2; 1M&1F-suburban)')
read_file.to_csv (r"C:\Users\20204113\OneDrive - TU Eindhoven\2_Research\1_Groceries\DATA\Groceries sorted\DATA GROCERIES-forpredict(hh3).csv", index = None, header=True)

In [3]:
# read_file = pd.read_excel (open (r"C:\Users\20204113\OneDrive - TU Eindhoven\2_Research\1_Groceries\DATA\Groceries sorted\DATA GROCERIES-forpredict(HH1-5).xlsx", 'rb'), sheet_name='H4 (4; 1M,1F&2K-urban)')
# read_file.to_csv (r"C:\Users\20204113\OneDrive - TU Eindhoven\2_Research\1_Groceries\DATA\Groceries sorted\DATA GROCERIES-forpredict(hh4).csv", index = None, header=True)

In [4]:
df = pd.read_csv (r"C:\Users\20204113\OneDrive - TU Eindhoven\2_Research\1_Groceries\DATA\Groceries sorted\DATA GROCERIES-forpredict(hh3).csv", parse_dates=[['date', 'timestamp']])

del df["HH"]
# df.describe(include='all')

## Recode variables

In [5]:
# new row for reordered items 
df['reorder'] = df.item_type.duplicated()

# convert reorder row to boolean value
df['reorder'] = df['reorder'].astype(bool)


#create an ID for each item 
df.rename(columns={'item_id':'item_name'}, inplace=True)
df['item_id'] = pd.factorize(df['item_name'])[0]

#create an ID for each type 
df['type_id'] = pd.factorize(df['item_type'])[0]


#first make a new column for the amount of items per order
df['order_amount'] = df.groupby('order_ID')['amount'].transform('sum')

# make a new column for the price per order
df['order_price'] = df.groupby('order_ID')['price_total'].transform('sum')

In [6]:
df['week'] = df['week'].astype(int)

df['order_ID'] = df['order_ID'].astype(int)

df['amount'] = df['amount'].astype(int)

df['promo'] = df['promo'].astype(bool)

df['item_id'] = df['item_id'].astype(int)

df['type_id'] = df['type_id'].astype(int)

In [7]:
df_des = df.describe(include='all', datetime_is_numeric=True)

df_styled = df_des.style.background_gradient() #adding a gradient based on values in cell
# dfi.export(df_styled,"describeHH2.png")

In [8]:
df.dtypes

date_timestamp    datetime64[ns]
week                       int32
order_ID                   int32
item_name                 object
amount                     int32
price_unit               float64
price_total              float64
item_type                 object
category                  object
day                       object
time                      object
store_type                object
store_name                object
promo                       bool
reorder                     bool
item_id                    int32
type_id                    int32
order_amount               int64
order_price              float64
dtype: object

### New (numeric) variables for the confusion matrix

In [9]:
# Recode dow

df['day_num']=df['day'].astype('category').cat.codes

In [10]:
# Recode store_type

df['storetype_num']=df['store_type'].astype('category').cat.codes

In [11]:
# Recode store_name

df['storename_num']=df['store_name'].astype('category').cat.codes

In [12]:
# Recode categories

df['cat_num']=df['category'].astype('category').cat.codes

In [13]:
# Recode time

df['time_num']=df['time'].astype('category').cat.codes

In [14]:
# Recode time

df['promo_num']=df['promo'].astype('category').cat.codes

In [15]:
df.dtypes

date_timestamp    datetime64[ns]
week                       int32
order_ID                   int32
item_name                 object
amount                     int32
price_unit               float64
price_total              float64
item_type                 object
category                  object
day                       object
time                      object
store_type                object
store_name                object
promo                       bool
reorder                     bool
item_id                    int32
type_id                    int32
order_amount               int64
order_price              float64
day_num                     int8
storetype_num               int8
storename_num               int8
cat_num                     int8
time_num                    int8
promo_num                   int8
dtype: object

#### convert time and date columns

In [16]:
df['datetime'] = pd.to_datetime(df.date_timestamp)
del df["date_timestamp"]

In [17]:
df['timestamp']=pd.to_numeric(df['datetime'])

In [18]:
df['dates'] = pd.to_datetime(df['datetime']).dt.date
df['times'] = pd.to_datetime(df['datetime']).dt.time

In [19]:
df['times_min'] = df.datetime.dt.hour *100 + df.datetime.dt.minute
# df['dates_hours']=df['dates'].astype('category').cat.codes

df['dates_days']=df.datetime.dt.year * 10000 + df.datetime.dt.month * 100 + df.datetime.dt.day

In [20]:
# df[['datetime', 'timestamp', 'dates', 'times', 'dates_num', 'times_num']].head()
df[['datetime', 'timestamp', 'dates', 'times', 'times_min', 'dates_days']].head()

,datetime,timestamp,dates,times,times_min,dates_days
0,2021-11-23 18:41:00,1637692860000000000,2021-11-23,18:41:00,1841,20211123
1,2021-11-23 18:41:00,1637692860000000000,2021-11-23,18:41:00,1841,20211123
2,2021-11-23 18:41:00,1637692860000000000,2021-11-23,18:41:00,1841,20211123
3,2021-11-23 18:41:00,1637692860000000000,2021-11-23,18:41:00,1841,20211123
4,2021-11-23 18:41:00,1637692860000000000,2021-11-23,18:41:00,1841,20211123


In [21]:
df.dtypes

week                      int32
order_ID                  int32
item_name                object
amount                    int32
price_unit              float64
price_total             float64
item_type                object
category                 object
day                      object
time                     object
store_type               object
store_name               object
promo                      bool
reorder                    bool
item_id                   int32
type_id                   int32
order_amount              int64
order_price             float64
day_num                    int8
storetype_num              int8
storename_num              int8
cat_num                    int8
time_num                   int8
promo_num                  int8
datetime         datetime64[ns]
timestamp                 int64
dates                    object
times                    object
times_min                 int64
dates_days                int64
dtype: object

## Save the final dataframe as a new csv file

In [22]:
df.head()

,week,order_ID,item_name,amount,price_unit,price_total,item_type,category,day,time,...,storename_num,cat_num,time_num,promo_num,datetime,timestamp,dates,times,times_min,dates_days
0,1,1,HAMBURGERBROODJES 6X55,1,0.99,0.99,sandwiches,bakery,Tuesday,evening,...,1,0,1,0,2021-11-23 18:41:00,1637692860000000000,2021-11-23,18:41:00,1841,20211123
1,1,1,WITTE BAGUETTEN/BROODJ,1,0.39,0.39,sandwiches,bakery,Tuesday,evening,...,1,0,1,0,2021-11-23 18:41:00,1637692860000000000,2021-11-23,18:41:00,1841,20211123
2,1,1,KEIZERBROODJES 4ST,3,0.69,2.07,sandwiches,bakery,Tuesday,evening,...,1,0,1,0,2021-11-23 18:41:00,1637692860000000000,2021-11-23,18:41:00,1841,20211123
3,1,1,PITABROODJES 4X80G,1,0.99,0.99,pita bread,bakery,Tuesday,evening,...,1,0,1,0,2021-11-23 18:41:00,1637692860000000000,2021-11-23,18:41:00,1841,20211123
4,1,1,PANNENKOEKENMIX 446,1,2.99,2.99,pancake mix,bakery,Tuesday,evening,...,1,0,1,0,2021-11-23 18:41:00,1637692860000000000,2021-11-23,18:41:00,1841,20211123


In [23]:
df.to_csv(r"C:\Users\20204113\OneDrive - TU Eindhoven\2_Research\1_Groceries\DATA\9th week - narrative (3rd attempt)\HH3\df\df_HH3.csv", index=False)

## What stores and categories?

In [24]:
# Find unique values of a column
print(df['store_type'].unique())

['supermarket' 'bakery' 'pet supplies']


In [25]:
# Find unique values for store names
print(df['store_name'].unique())

['Aldi' 'Wouters' 'Colruyt' 'Vanessa' 'Tine' 'Albert Heijn' 'Delhaize'
 'Food Factory' 'Tom&Co' 'Maxi Zoo']


In [26]:
# Find unique values of a column
print(df['category'].unique())

['bakery' 'beverages' 'breakfast & spreads' 'canned foods'
 'care & hygiene' 'condiments' 'cooking aids' 'dairy & plant based'
 'frozen foods' 'fruit & vegetables' 'household' 'meat, fish & veggie'
 'non-food' 'pasta & rice' 'snacks']


## [old] Correlation Matrix

In [27]:
# Recode day

def day_to_numeric(x):
    if x=='Monday': return 1
    if x=='Tuesday':   return 2
    if x=='Wednesday':   return 3
    if x=='Thursday':   return 4
    if x=='Friday':   return 5
    if x=='Saturday':   return 6   
    if x=='Sunday':   return 7    
    
df['day_num'] = df['day'].apply(day_to_numeric)

In [28]:
# Recode store_type

def storetype_to_numeric(x):
    if x=='supermarket': return 1
    if x=='bakery':   return 3
    if x=='pet supplies': return 4

df['storetype_num'] = df['store_type'].apply(storetype_to_numeric)

In [29]:
# Recode store_name

def storename_to_numeric(x):
    if x=='Carrefour': return 1
    if x=='Okay':   return 2
    if x=='Albert Heijn':   return 3
    if x=='Delhaize':   return 4
    if x=='Sys':   return 5
    if x=='Brabo':   return 6
    if x=='Versavel Poelman':   return 7
    if x=='Ikea':   return 8
    if x=='Kruidvat':   return 9
    if x=='Aldi':   return 10
    if x=='Vanessa':   return 11
    if x=='Tine':   return 12
    if x=='Food Factory':   return 13
    if x=='Maxi Zoo':   return 14
    if x=='MCD':   return 15
    if x=='De Pers':   return 16
    if x=='Jumbo':   return 17
    if x=='Colruyt':   return 18
    if x=='Spar':   return 19

df['storename_num'] = df['store_name'].apply(storename_to_numeric)

In [30]:
# Recode time

def time_to_numeric(x):
    if x=='morning': return 1
    if x=='noon':   return 2
    if x=='afternoon':   return 3
    if x=='evening':   return 4

df['time_num'] = df['time'].apply(time_to_numeric)

In [31]:
# Recode categories

def cat_to_numeric(x):
    if x=='nuts, seeds & dried fruit': return 1
    if x=='bakery':   return 2
    if x=='bakery ':   return 2
    if x=='beverages':   return 3
    if x=='dairy & plant based':   return 4
    if x=='breakfast & spreads':   return 5
    if x=='snacks':   return 6
    if x=='non-food':   return 7
    if x=='fruit & vegetables':   return 8
    if x=='meat, fish & veggie':   return 9
    if x=='cooking aids':   return 10
    if x=='cooking aid':   return 10
    if x=='canned foods':   return 11
    if x=='household':   return 12
    if x=='condiments':   return 13
    if x=='care & hygiene':   return 14
    if x=='care & hyygiene':   return 14
    if x=='frozen foods':   return 15
    if x=='pasta & rice':   return 16

df['cat_num'] = df['category'].apply(cat_to_numeric)

In [32]:
# df['time_num'] = df['time_num'].astype(int)

# df['day_num'] = df['day_num'].astype(int)

# df['cat_num'] = df['cat_num'].astype(int)

# df['storename_num'] = df['storename_num'].astype(int)

# df['storetype_num'] = df['storetype_num'].astype(int)

# df.dtypes

In [33]:
# use the corr function to display the correlation between all the features
data_corr = df.corr()
data_corr

,week,order_ID,amount,price_unit,price_total,promo,reorder,item_id,type_id,order_amount,order_price,day_num,storetype_num,storename_num,cat_num,time_num,promo_num,timestamp,times_min,dates_days
week,1.000000,0.983964,0.000475,0.122041,0.107309,0.092026,0.344865,0.061720,0.003828,-0.102348,0.126743,0.161367,-0.005322,0.367077,0.065267,-0.118031,0.092026,0.990564,-0.216324,0.915499
order_ID,0.983964,1.000000,0.013224,0.118954,0.106284,0.089096,0.368181,0.041209,-0.017346,-0.151575,0.072409,0.211144,0.054680,0.299612,0.030170,-0.171858,0.089096,0.965929,-0.275264,0.869815
amount,0.000475,0.013224,1.000000,-0.126146,0.323628,-0.015742,-0.013644,-0.275626,-0.298109,0.066612,0.030944,0.125002,0.090912,0.103206,-0.239086,-0.021257,-0.015742,0.005617,-0.051281,-0.000019
price_unit,0.122041,0.118954,-0.126146,1.000000,0.873330,0.070597,-0.090478,0.147591,0.160780,-0.104253,0.020674,0.079382,0.116867,-0.007062,0.228717,-0.040417,0.070597,0.115063,-0.014695,0.087577
price_total,0.107309,0.106284,0.323628,0.873330,1.000000,0.056399,-0.083007,0.016650,0.016410,-0.052196,0.044916,0.108029,0.127349,0.050788,0.097157,-0.034415,0.056399,0.102503,-0.017640,0.076451
promo,0.092026,0.089096,-0.015742,0.070597,0.056399,1.000000,-0.060460,0.043605,0.041315,0.043451,0.085178,-0.012927,-0.039748,0.055967,-0.026264,0.008595,1.000000,0.081269,0.012676,0.062623
reorder,0.344865,0.368181,-0.013644,-0.090478,-0.083007,-0.060460,1.000000,0.089120,0.028554,-0.042382,-0.037650,-0.037782,-0.035132,0.114939,-0.190556,-0.087849,-0.060460,0.318552,-0.145627,0.262625
item_id,0.061720,0.041209,-0.275626,0.147591,0.016650,0.043605,0.089120,1.000000,0.992444,0.174348,0.216430,-0.222159,-0.181337,0.116691,0.402244,0.245562,0.043605,0.057101,0.280509,0.078416
type_id,0.003828,-0.017346,-0.298109,0.160780,0.016410,0.041315,0.028554,0.992444,1.000000,0.173601,0.211113,-0.234212,-0.182361,0.099104,0.418544,0.250451,0.041315,0.000408,0.299116,0.027760
order_amount,-0.102348,-0.151575,0.066612,-0.104253,-0.052196,0.043451,-0.042382,0.174348,0.173601,1.000000,0.898929,-0.416373,-0.539993,0.600981,0.039129,0.870932,0.043451,-0.087767,0.821047,0.007184


In [34]:
df_corr = data_corr

df_styled = df_corr.style.background_gradient() #adding a gradient based on values in cell
dfi.export(df_styled,"confmatrixHH4.png")

SyntaxError: not a PNG file (<string>)

### Interesting correlations

In [ ]:
# Is there a correlation between day of the week & supermarket type?

# get the Pearson correlation between both variables
current_corr = round(data_corr["day_num"]["storetype_num"], 3)

# select the x and y features
x = df['day_num'] # select the x feature
y = df['storetype_num'] # select the y feature

# fit the points with the scatter function
plt.plot(x, y, 'o',color='orange');


# give the graph a title and labels
plt.xlabel("Day")
plt.ylabel('store')
plt.title(f"The correlation between day and store type (r={current_corr})")

# call the graph (show function)
plt.grid(True)
plt.show()

In [ ]:
# Is there a correlation between day of the week & supermarket type? + TIME


import math

# Get pairwise correlation coefficients
cor = df.corr()

# Independent variables
x = 'day_num'
y = 'time_num'

# Dependent variable
z = 'storetype_num'

# Pairings
xz = cor.loc[ x, z ]
yz = cor.loc[ y, z ]
xy = cor.loc[ x, y ]

Rxyz = math.sqrt((abs(xz**2) + abs(yz**2) - 2*xz*yz*xy) / (1-abs(xy**2)) )
R2 = Rxyz**2

# Calculate adjusted R-squared
n = len(df) # Number of rows
k = 2       # Number of independent variables
R2_adj = 1 - ( ((1-R2)*(n-1)) / (n-k-1) )

print(f'Storetype is {R2_adj} percent dependent on day and time')

In [ ]:
# Is there a correlation between day of the week & supermarket NAME?

# get the Pearson correlation between both variables
current_corr = round(data_corr["day_num"]["storename_num"], 3)

# select the x and y features
x = df['day_num'] # select the x feature
y = df['storename_num'] # select the y feature

# fit the points with the scatter function
plt.plot(x, y, 'o',color='red');


# give the graph a title and labels
plt.xlabel("Day")
plt.ylabel('Store name')
plt.title(f"The correlation between day and store NAME (r={current_corr})")

# call the graph (show function)
plt.grid(True)
plt.show()

In [ ]:
# Is there a correlation between DAY + NAME + TIME

import math

# Get pairwise correlation coefficients
cor = df.corr()

# Independent variables
x = 'time_num'
y = 'storename_num'

# Dependent variable
z = 'day_num'

# Pairings
xz = cor.loc[ x, z ]
yz = cor.loc[ y, z ]
xy = cor.loc[ x, y ]

Rxyz = math.sqrt((abs(xz**2) + abs(yz**2) - 2*xz*yz*xy) / (1-abs(xy**2)) )
R2 = Rxyz**2

# Calculate adjusted R-squared
n = len(df) # Number of rows
k = 2       # Number of independent variables
R2_adj = 1 - ( ((1-R2)*(n-1)) / (n-k-1) )



## Time based off day and store
xt = 'day_num'
yt = 'storename_num'
zt = 'time_num'
xzt = cor.loc[ xt, zt]
yzt = cor.loc[ yt, zt ]
xyt = cor.loc[ xt, yt ]

Rxyzt = math.sqrt((abs(xzt**2) + abs(yzt**2) - 2*xzt*yzt*xyt) / (1-abs(xyt**2)) )
R2t = Rxyzt**2

nt = len(df) # Number of rows
kt = 2       # Number of independent variables
R2_adjt = 1 - ( ((1-R2t)*(n-1)) / (n-k-1) )



## Store based of day and time
xs = 'day_num'
ys = 'time_num'
zs = 'storename_num'
xzs = cor.loc[ xs, zs ]
yzs = cor.loc[ ys, zs ]
xys = cor.loc[ xs, ys ]

Rxyzs = math.sqrt((abs(xzs**2) + abs(yzs**2) - 2*xzs*yzs*xys) / (1-abs(xys**2)) )
R2s = Rxyzs**2

ns = len(df) # Number of rows
ks = 2       # Number of independent variables
R2_adjs = 1 - ( ((1-R2s)*(n-1)) / (n-k-1) )


print(f'Day is {R2_adj*100} percent dependent on time and store')
print(f'Time is {R2_adjt*100} percent dependent on day and store')
print(f'Storename is {R2_adjs*100} percent dependent on day and time')